In [18]:
import pandas as pd
from keras.models import load_model
import numpy as np

class TickerReader(object):
    def __init__(self, filename):
        self.df = pd.read_csv(filename)
        self.df = self.df.drop(columns=['open', 'high', 'low', 'close', 'volume', 'dividend', 'split'])
        self.df = self.df.head(10000).sort_values(by=['time']).groupby(['id'])
        self.model = load_model('test.h5')
        
    def predict(self):
        ids = []
        for i, df in list(self.df):
#             TODO: get row count
            x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'adjusted_open']].values
            if x.shape[0] >= 60:
                ids.append([df['id'], df['time']])

        x_input = np.empty((len(ids), 60, 6), dtype=np.float32)

        current_point = 0

        for i, df in list(self.df):
            df['moving_average'] = df['adjusted_close'].rolling(5).mean()
            x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'moving_average']].values
            if x.shape[0] >= 60:
                x_input[current_point, :, :] = x[-60:, :]
            current_point += 1
            
        y = model.predict(x_input)
        return y

In [19]:
reader = TickerReader('/Users/eureyuri/Desktop/School/Brandeis/Computer-Science/149B/cosi-149b-project2/training_project2.csv')

In [20]:
from keras.models import load_model

model = load_model('test.h5')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [26]:
for i, df in list(reader.df):
    x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'adjusted_open']].values
    if x.shape[0] >= 60:
        x = x[-60:, :]
        print(x.shape)
        x = x.reshape((1, 60, 6))
        y = model.predict(x)
        print(y)

(60, 6)
[[ 0.08192189  0.122981    0.24014714  0.2754598  -0.4026155  -0.05310421]]
(60, 6)
[[ 0.08192189  0.122981    0.24014714  0.2754598  -0.4026155  -0.05310421]]
(60, 6)
[[ 0.08192189  0.122981    0.24014714  0.2754598  -0.4026155  -0.05310421]]
(60, 6)
[[ 0.08192189  0.122981    0.24014714  0.2754598  -0.4026155  -0.05310421]]


In [35]:
list(reader.df)[0][1]

,adjusted_open,adjusted_high,adjusted_low,adjusted_close,adjusted_volume,time,id
0,29.092421,29.133072,26.971804,27.357987,10897100.0,1999-11-19,0
1,27.988074,29.810585,27.141182,29.810585,4705200.0,1999-11-22,0
2,28.794315,29.559905,27.269910,27.269910,4274400.0,1999-11-23,0
3,27.188608,28.414907,27.100532,27.818696,3464400.0,1999-11-24,0
4,27.696743,28.116802,27.608667,27.906772,1237100.0,1999-11-26,0
5,27.778045,28.753664,27.479939,28.543635,2914700.0,1999-11-29,0
6,28.455558,29.092421,27.737394,28.584286,3083000.0,1999-11-30,0
7,28.584286,29.431177,28.374257,29.092421,2115400.0,1999-12-01,0
8,29.641207,30.488098,29.261799,29.898662,2195900.0,1999-12-02,0
9,30.447447,30.955582,30.020614,30.149341,2175700.0,1999-12-03,0


In [42]:
import numpy as np
from datetime import datetime, timedelta

ids = []
for i, df in list(reader.df):
    x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'adjusted_open']].values
    if x.shape[0] >= 60:
        ids.append([i, datetime.strptime(df['time'].iloc[-1], '%Y-%m-%d')])
print(ids)

x_input = np.empty((len(ids), 60, 6), dtype=np.float32)

current_point = 0

for i, df in list(reader.df):
    df['moving_average'] = df['adjusted_close'].rolling(5).mean()
    x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'moving_average']].values
    if x.shape[0] >= 60:
        x_input[current_point, :, :] = x[-60:, :]
    current_point += 1
    print(x)
# print(x_input)
# y = model.predict(x_input)
# y

[[0, datetime.datetime(2017, 12, 29, 0, 0)], [1, datetime.datetime(2017, 12, 29, 0, 0)], [2, datetime.datetime(2017, 12, 29, 0, 0)], [3, datetime.datetime(2014, 8, 7, 0, 0)]]
[[2.90924207e+01 2.91330715e+01 2.69718041e+01 2.73579867e+01
  1.08971000e+07            nan]
 [2.79880741e+01 2.98105848e+01 2.71411825e+01 2.98105848e+01
  4.70520000e+06            nan]
 [2.87943149e+01 2.95599049e+01 2.72699100e+01 2.72699100e+01
  4.27440000e+06            nan]
 ...
 [6.67442873e+01 6.69925550e+01 6.66847030e+01 6.68336636e+01
  1.03858500e+06 6.68972202e+01]
 [6.69726935e+01 6.70124164e+01 6.65556038e+01 6.69826243e+01
  6.07022000e+05 6.69071509e+01]
 [6.71814163e+01 6.72610388e+01 6.66141066e+01 6.66539178e+01
  1.06489500e+06 6.68275066e+01]]
[[2.2100000e+01 2.3550000e+01 2.1780000e+01 2.3000000e+01 3.2216510e+07
            nan]
 [2.2670000e+01 2.3265000e+01 2.2635800e+01 2.2910000e+01 1.0755328e+07
            nan]
 [2.2960000e+01 2.4349300e+01 2.2940000e+01 2.4150000e+01 8.0830970e+06

In [44]:
(datetime.strptime('2019-01-01', '%Y-%m-%d') + timedelta(days=7)).strftime('%Y-%m-%d')

'2019-01-08'

In [17]:
df = reader.df.head(10000).sort_values(by=['time']).groupby(['id'])
list(df)[1][1]

,open,high,low,close,volume,dividend,split,adjusted_open,adjusted_high,adjusted_low,adjusted_close,adjusted_volume,time,id
4557,22.100,23.5500,21.7800,23.00,32216510.0,0.0,1.0,22.100,23.5500,21.7800,23.00,32216510.0,2016-11-01,1
4558,22.670,23.2650,22.6358,22.91,10755328.0,0.0,1.0,22.670,23.2650,22.6358,22.91,10755328.0,2016-11-02,1
4559,22.960,24.3493,22.9400,24.15,8083097.0,0.0,1.0,22.960,24.3493,22.9400,24.15,8083097.0,2016-11-03,1
4560,23.820,25.5000,23.5600,25.20,6878500.0,0.0,1.0,23.820,25.5000,23.5600,25.20,6878500.0,2016-11-04,1
4561,25.540,26.4100,24.8900,25.08,8377025.0,0.0,1.0,25.540,26.4100,24.8900,25.08,8377025.0,2016-11-07,1
4562,24.300,26.5900,24.3000,26.40,6469979.0,0.0,1.0,24.300,26.5900,24.3000,26.40,6469979.0,2016-11-08,1
4563,26.830,29.1400,26.0000,28.72,10675856.0,0.0,1.0,26.830,29.1400,26.0000,28.72,10675856.0,2016-11-09,1
4564,29.080,29.7000,28.3000,29.10,8189887.0,0.0,1.0,29.080,29.7000,28.3000,29.10,8189887.0,2016-11-10,1
4565,29.180,29.9900,27.6300,29.30,5337239.0,0.0,1.0,29.180,29.9900,27.6300,29.30,5337239.0,2016-11-11,1
4566,29.500,29.9400,28.6100,29.62,5156737.0,0.0,1.0,29.500,29.9400,28.6100,29.62,5156737.0,2016-11-14,1


In [ ]:
from collections import default_dict
dictionary = default_dict(lambda: [])

for i, row in reader.df.sort_values(by=['time']).iterrows():
    dictionary[row['id']].append()

In [7]:
for i, row in reader.df.head(1).iterrows():
    print(row[['id', 'time']].values)
# adjusted_open	adjusted_high	adjusted_low	adjusted_close	adjusted_volume	time	

[0 '1999-11-19']
